In [3]:
from keras.datasets import mnist
from keras.utils import np_utils
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Training Parameters\n",
batch_size = 64
epochs = 15

# loads the MNIST dataset\n",
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns\n",
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras\n",
# We need to add a 4th dimenion to our date thereby changing our\n",
# Our original image shape of (60000,28,28) to (60000,28,28,1)\n",
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image \n",
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type\n",
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)\n",
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Now we one hot encode outputs\n",
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Let's count the number columns in our hot encoded matrix \n",
print ("Number of Classes: " + str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

# create model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

print(model.summary())
checkpoint = ModelCheckpoint("../Trained Models/MNIST_Checkpoint.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
callbacks = [checkpoint]

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Number of Classes: 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_______________________________________________

In [4]:
history = model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 2,
          callbacks = callbacks,
          validation_data = (x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
 - 23s - loss: 0.2108 - acc: 0.9352 - val_loss: 0.0514 - val_acc: 0.9831

Epoch 00001: val_loss improved from inf to 0.05142, saving model to ../Trained Models/MNIST_Checkpoint.h5
Epoch 2/15
 - 22s - loss: 0.0813 - acc: 0.9760 - val_loss: 0.0417 - val_acc: 0.9864

Epoch 00002: val_loss improved from 0.05142 to 0.04170, saving model to ../Trained Models/MNIST_Checkpoint.h5
Epoch 3/15
 - 22s - loss: 0.0638 - acc: 0.9817 - val_loss: 0.0360 - val_acc: 0.9878

Epoch 00003: val_loss improved from 0.04170 to 0.03600, saving model to ../Trained Models/MNIST_Checkpoint.h5
Epoch 4/15
 - 23s - loss: 0.0519 - acc: 0.9846 - val_loss: 0.0324 - val_acc: 0.9892

Epoch 00004: val_loss improved from 0.03600 to 0.03241, saving model to ../Trained Models/MNIST_Checkpoint.h5
Epoch 5/15
 - 22s - loss: 0.0459 - acc: 0.9861 - val_loss: 0.0294 - val_acc: 0.9905

Epoch 00005: val_loss improved from 0.03241 to 0.02945, saving model to ../Trained Models

In [5]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor = 'val_loss', # value being monitored for improvement\n",
                          min_delta = 0, #Abs value and is the min change required before we stop\n",
                          patience = 3, #Number of epochs we wait before stopping \n",
                          verbose = 1,
                          restore_best_weights = True) #keeps the best weigths once stopped\n",
# we put our call backs into a callback list\n",
callbacks = [earlystop, checkpoint]

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=64,
          epochs=3,
          verbose=2,
          callbacks = callbacks,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
 - 28s - loss: 0.0288 - acc: 0.9915 - val_loss: 0.0313 - val_acc: 0.9913

Epoch 00001: val_loss did not improve from 0.02614
Epoch 2/3
